In [2]:
from PyPDF2 import PdfReader
import pandas as pd
import re

In [3]:
def extract_text_from_pdf(file_path):
    with open(file_path, "rb") as file:
        pdf_reader = PdfReader(file)
        num_pages = len(pdf_reader.pages)

        text = ""
        for page_number in range(num_pages):
            page = pdf_reader.pages[page_number]
            text += page.extract_text()

        return text

In [4]:
# Provide the path to your PDF file
pdf_file_path = "Willows_of_Potomac_at_Eldwick_Program_Week_1A_2023.pdf"

# Call the function to extract text from the PDF
extracted_text = extract_text_from_pdf(pdf_file_path)

# Print the extracted text
print(extracted_text)

MCSL - L - 1 HY-TEK's MEET MANAGER 8.0 - 1:26 PM  6/16/2023  Page 1
Willows of Potomac at Eldwick
06/17/2023
Meet Program - All Events
Event  1   Boys 12 & Under 100 SC Meter IM
MCSL: 1:03.31 7/24/2021 Andrew Vanas
Age Lane   Team Name Seed Time
Heat   1 of 1   Finals
WillowsofPotomac-PV 1:38.68   9 Yu, Bernard1 _____
Eldwick Sharks 1:52.24  11 Smith, Ethan2 _____
WillowsofPotomac-PV 1:33.88  11 Hsu, Isaac3 _____
Eldwick Sharks 1:45.41  12 Liu, Chase4 _____
WillowsofPotomac-PV 1:34.34  12 Zeng, Reid5 _____
Eldwick Sharks NT  11 Echeverri, Bruno6 _____
Event  2   Girls 12 & Under 100 SC Meter IM
MCSL: 1:06.59 7/22/2017 Carly Sebring
Age Lane   Team Name Seed Time
Heat   1 of 1   Finals
WillowsofPotomac-PV 1:44.59  12 Kim, Olivia1 _____
Eldwick Sharks 1:28.50  12 Lee, Georgiana2 _____
WillowsofPotomac-PV 1:28.50  12 Lin, Fiona3 _____
Eldwick Sharks 1:18.03  12 Chen, Alexa4 _____
WillowsofPotomac-PV 1:34.51  12 Lee, Kyla5 _____
Eldwick Sharks 1:47.90  12 Rosenfeld, Julia6 _____
Event  3  

In [10]:
# Team names as variables
team1 = "WillowsofPotomac-PV" #"FV-PV"
team2 = "Eldwick Sharks" #"RV-PV"

#assign text
text = extracted_text  # paste your text here

#this line is for cleaning up misc text tha screwing up the processing
#replace all star with new line
text = text.replace("ALL*", "\n")
#replace blank lines after seed time with new line
text = text.replace("_____", "\n")
#replace blank lines after seed time with new line
text = text.replace("Eldwick Sharks", "EldwickSharks")


# Add a newline character before every "#"
text = text.replace("#", "\n#")
# print(text)

# Split the text into lines
lines = text.split("\n")

# Prepare a list to hold dictionaries, each representing a swimmer
swimmers = []
current_event = None

for line in lines:
    # Check if the line starts a new event, some program starts with # some with Event
    if line.startswith("#") or line.startswith("Event"):
        current_event = line
        #print(current_event)

    # Check if the line describes a swimmer
    elif re.match(f"^({team1}|{team2})", line):        
        parts = line.split()

        # Construct a dictionary for the swimmer
        swimmer = {
            "Event": current_event,
            "Team": parts[0][:2],
            "Time": parts[1],
            "Age": parts[2],
            "Name": " ".join(parts[3:])
        }
        # Append the dictionary to the list
        swimmers.append(swimmer)

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(swimmers)

# # Display the DataFrame
# # print(df)


In [11]:
df.head()

,Event,Team,Time,Age,Name
0,Event 1 Boys 12 & Under 100 SC Meter IM,Wi,1:38.68,9,"Yu, Bernard1"
1,Event 1 Boys 12 & Under 100 SC Meter IM,El,Sharks,1:52.24,"11 Smith, Ethan2"
2,Event 1 Boys 12 & Under 100 SC Meter IM,Wi,1:33.88,11,"Hsu, Isaac3"
3,Event 1 Boys 12 & Under 100 SC Meter IM,El,Sharks,1:45.41,"12 Liu, Chase4"
4,Event 1 Boys 12 & Under 100 SC Meter IM,Wi,1:34.34,12,"Zeng, Reid5"


In [13]:
# Function to convert time to seconds for sorting
def time_to_seconds(time):
    if ":" in time:
        minutes, seconds = time.split(":")
        return int(minutes) * 60 + float(seconds)
    else:
        return float(time)

In [14]:
def remove_parentheses(df, column_name):
    df[column_name] = df[column_name].str.replace(")", "", regex=False)
    return df

# Use the function
df = remove_parentheses(df, "Event")

In [15]:
# Convert time to seconds
df["TimeInSeconds"] = df["Time"].apply(time_to_seconds)

# Sort by event and time
df = df.sort_values(["Event", "TimeInSeconds"])

# Assign ranks within each event group
df["Rank"] = df.groupby("Event")["TimeInSeconds"].rank()

# Scoring system
scores = {1.0: 6, 2.0: 4, 3.0: 3, 4.0: 2, 5.0: 1}

# Assign scores based on ranks
df["Score"] = df["Rank"].map(scores).fillna(0)

# Drop auxiliary columns
df = df.drop(["TimeInSeconds", "Rank"], axis=1)

ValueError: could not convert string to float: 'Sharks'

In [12]:
df.to_csv("swimmers.csv", index=False)

In [76]:
# Calculate total points by team
team_scores = df.groupby("Team")["Score"].sum()

# Export the team scores to a CSV file
team_scores.to_csv("team_scores.csv")